# Get PASTIS modes

## --- LUVOIR A --

We will now perform a singular value decomposition (SVD) on the PASTIS matrix to get the PASTIS modes and save them.

I started working on the stability calculations in here too, but I will expand on that (and do it properly) in notebook 11.

In [ ]:
# Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline
from astropy.io import fits
import astropy.units as u
import hcipy as hc
#from hcipy.optics.segmented_mirror import SegmentedMirror

os.chdir('../../pastis/')
#from e2e_simulators.luvoir_imaging import LuvoirAPLC

In [ ]:
# Which directory are we working in?
savedpath = '/Users/pueyo/Documents/data_from_repos/pastis_data/2020-05-07T10-55-02_luvoir-medium'   # large apodizer LUVOIR
savedpath2 = '/Users/pueyo/Documents/data_from_repos/pastis_data/2020-06-08T18-11-32_luvoir-medium'   # large apodizer LUVOIR

## Read the matrix

It's your choice wheter you read the analytical or numerical matrix.

In [ ]:
matrix = fits.getdata(os.path.join(savedpath, 'matrix_numerical', 'PASTISmatrix_num_piston_Noll1.fits'))
matrix2 = fits.getdata(os.path.join(savedpath2, 'matrix_numerical', 'PASTISmatrix_num_Multiple_Modes_MaxZer_1.fits'))

plt.figure(figsize=(10, 10))
plt.imshow(matrix)
plt.colorbar()

plt.figure(figsize=(10, 10))
plt.imshow(matrix2)
plt.colorbar()

## Get eigenmodes and eigenvalues

We should actually be using an SVD, which I do at the end of this notebook.

In [ ]:
evals, evecs = np.linalg.eig(matrix)

In [ ]:
# Hack to use SVD modes instead
#evals = s
#evecs = u

In [ ]:
print('evals.shape: {}'.format(evals.shape))
print('evals:\n{}'.format(evals))

In [ ]:
print('evecs.shape: {}'.format(evecs.shape))
#print('evecs:\n{}'.format(evecs))

Which dimension are the eigenvectors in?

We know

$$M \cdot u = \lambda \cdot u$$

so we can test this.

In [ ]:
# Evaluate left and right side of eigenvalue problem for mode number n
n = 17

left = np.dot(matrix, evals[n])
right = np.dot(matrix, evecs[:, n])

# Compare them
print('Are the left and right side the same:')
print(np.allclose(left, right))

### Sort from lowest to highest eigenvalue

In [ ]:
# Sort them
sorted_evals = np.sort(evals)
sorted_indices = np.argsort(evals)
sorted_evecs = evecs[:, sorted_indices]

print('Sorted evals:')
print(sorted_evals)
#np.savetxt(os.path.join(savedpath, 'results', 'eigenvalues.txt'), sorted_evals)
#np.savetxt(os.path.join(savedpath, 'results', 'eigenvectors.txt'), sorted_evecs)

plt.figure(figsize=(14, 8))
#plt.plot(evals, label='Unsorted from eigendecomposition')
plt.plot(sorted_evals, label='Sorted lowest to highest evals')
plt.semilogy()
plt.xlabel('Eigenmodes')
plt.ylabel('Log Eigenvalues')
plt.legend()
#plt.savefig(os.path.join(savedpath, 'results', 'LUVOIR_eigenvals.pdf'))

## Mode display

Instead of using the `SegmentedTelescopeAPLC` class to display the modes, which would require us to load andn create all the files needed for the APLC propagation, we will simply use a `SegmentedMirror`.

In [ ]:
# Load aperture files needed for SM
nseg = 120
wvln = 638e-9

datadir = '/Users/pueyo/PythonPackages/PASTIS/LUVOIR_delivery_May2019/'
aper_path = 'inputs/TelAp_LUVOIR_gap_pad01_bw_ovsamp04_N1000.fits'
aper_ind_path = 'inputs/TelAp_LUVOIR_gap_pad01_bw_ovsamp04_N1000_indexed.fits'
aper_read = hc.read_fits(os.path.join(datadir, aper_path))
aper_ind_read = hc.read_fits(os.path.join(datadir, aper_ind_path))

pupil_grid = hc.make_pupil_grid(dims=aper_ind_read.shape[0], diameter=15)
aper = hc.Field(aper_read.ravel(), pupil_grid)
aper_ind = hc.Field(aper_ind_read.ravel(), pupil_grid)

wf_aper = hc.Wavefront(aper, wvln)

# Load segment positions from fits header
hdr = fits.getheader(os.path.join(datadir, aper_ind_path))

poslist = []
for i in range(nseg):
    segname = 'SEG' + str(i+1)
    xin = hdr[segname + '_X']
    yin = hdr[segname + '_Y']
    poslist.append((xin, yin))
    
poslist = np.transpose(np.array(poslist))
seg_pos = hc.CartesianGrid(hc.UnstructuredCoords(poslist))

In [ ]:
#Full aperture to test zernikes
aper_full = hc.Field(np.ones([1000**2]), pupil_grid)
wf_aper_full = hc.Wavefront(aper_full, wvln)

In [ ]:
pupil_diameter = 15.0 #m actual circumscribed diameter, used for lam/D calculations other measurements normalized by this diameter
pupil_inscribed = 13.5 #m actual inscribed diameter
actual_segment_flat_diameter = 1.2225 #m actual segment flat-to-flat diameter
actual_segment_gap = 0.006 #m actual gap size between segments
spider_width = 0.150 #m actual strut size
lower_spider_angle = 12.7 #deg angle at which lower spiders are offset from vertical
spid_start = 0.30657 #m spider starting point distance from center of aperture
gap_padding = 1
segment_gap = actual_segment_gap * gap_padding
segment_flat_diameter = actual_segment_flat_diameter - (segment_gap - actual_segment_gap)
segment_circum_diameter = 2 / np.sqrt(3) * segment_flat_diameter #segment circumscribed diameter

In [ ]:
# Making a segment
segment = hc.hexagonal_aperture(segment_circum_diameter, np.pi / 2)
segment_sampled = hc.evaluate_supersampled(segment, pupil_grid, 1)
plt.figure(figsize=(10, 10))
hc.imshow_field(segment_sampled)

In [ ]:
seg_pos.coords.coords[0]

In [ ]:
# Scaling the positions for LUVOIR segments
seg_pos = seg_pos.scaled(pupil_diameter)
aper2, segs2 = hc.make_segmented_aperture(segment,seg_pos, segment_transmissions=1, return_segments= True)
luvoir_segmented_pattern = hc.evaluate_supersampled(aper2, pupil_grid, 1)
seg_evaluated = []
for seg_tmp in segs2:
    tmp_evaluated = hc.evaluate_supersampled(seg_tmp, pupil_grid, 1)
    seg_evaluated.append(tmp_evaluated)
plt.figure(figsize=(10, 10))
hc.imshow_field(luvoir_segmented_pattern)

In [ ]:
#Comparing result with generic LUVOIR function in HCIPy
aper, segments = hc.aperture.make_luvoir_a_aperture(normalized=False, 
                                                    with_spiders=False, with_segment_gaps=True, gap_padding=1,
                                                    segment_transmissions=1, return_header=False, 
                                                    return_segments=True)
segments0 = hc.evaluate_supersampled(aper, pupil_grid, 1)
plt.figure(figsize=(10, 10))
hc.imshow_field(segments0)
print(np.max(np.abs(luvoir_segmented_pattern - segments0)))
plt.figure(figsize=(10, 10))
hc.imshow_field(luvoir_segmented_pattern-segments0)

In [ ]:
# Creating a modal basis with pistons on each segment
mode_basis_segments = hc.ModeBasis(seg_evaluated, pupil_grid)
segment_piston_aberrations = hc.optics.DeformableMirror(mode_basis_segments)
pistons = np.random.normal(np.pi, size=test.num_modes)
#pistons = np.zeros([test.num_modes])
segment_piston_aberrations.actuators = pistons
post_piston_aberrations_field = segment_piston_aberrations(wf_aper)
plt.figure(figsize=(10, 10))
hc.imshow_field(decul.phase)
plt.figure(figsize=(10, 10))
hc.imshow_field(decul.amplitude)

In [ ]:
# Creating a modal basis with global zernikes
Nzernike = 12
mode_basis_global_zernike = hc.mode_basis.make_zernike_basis(12, pupil_inscribed, pupil_grid, starting_mode=1,
                                 ansi=False, radial_cutoff=True, use_cache=True)
global_zernike_aberrations = hc.optics.DeformableMirror(mode_basis_global_zernike)
zernike_coeffs = np.zeros([mode_basis_global_zernike.num_modes])
zernike_order = 5
zernike_coeffs[zernike_order] = 0.0001*wvln
global_zernike_aberrations.actuators = zernike_coeffs
zernike_aberrations = global_zernike_aberrations(wf_aper)
plt.figure(figsize=(10, 10))
hc.imshow_field(zernike_aberrations.phase)
plt.figure(figsize=(10, 10))
hc.imshow_field(zernike_aberrations.amplitude)

In [ ]:
for qq in range(0,Nzernike):
    mode_basis_global_zernike._transformation_matrix[:,qq] = seg_evaluated[seg_num]*mode_basis_global_zernike._transformation_matrix[:,qq]

In [ ]:
# Creating a modal basis with zernikes on each segment 
Nzernike = 3
seg_num = 0
mode_basis_global_zernike = hc.mode_basis.make_zernike_basis(Nzernike, segment_circum_diameter, pupil_grid.shifted(-seg_pos[seg_num]), starting_mode=1,
                                 ansi=False, radial_cutoff=True, use_cache=True)
for qq in range(0,Nzernike):
     mode_basis_global_zernike._transformation_matrix[:,qq] = seg_evaluated[seg_num]*mode_basis_global_zernike._transformation_matrix[:,qq]
for seg_num in range(1,120):
    print(seg_num)
    mode_basis_global_zernike_tmp = hc.mode_basis.make_zernike_basis(Nzernike, segment_circum_diameter, pupil_grid.shifted(-seg_pos[seg_num]), starting_mode=1,
                                 ansi=False, radial_cutoff=True, use_cache=True)
    for qq in range(0,Nzernike):
        mode_basis_global_zernike_tmp._transformation_matrix[:,qq] = seg_evaluated[seg_num]*mode_basis_global_zernike_tmp._transformation_matrix[:,qq]
    mode_basis_global_zernike.extend(mode_basis_global_zernike_tmp)
global_zernike_aberrations = hc.optics.DeformableMirror(mode_basis_global_zernike)
zernike_coeffs = np.zeros([mode_basis_global_zernike.num_modes])
zernike_order = 99
zernike_coeffs[zernike_order] = 0.0001*wvln
global_zernike_aberrations.actuators = zernike_coeffs
zernike_aberrations = global_zernike_aberrations(wf_aper)
plt.figure(figsize=(10, 10))
hc.imshow_field(zernike_aberrations.phase)
plt.figure(figsize=(10, 10))
hc.imshow_field(seg_evaluated[seg_num])

In [ ]:
global_zernike_aberrations = hc.optics.DeformableMirror(mode_basis_global_zernike)
zernike_coeffs = np.zeros([mode_basis_global_zernike.num_modes])
zernike_order = 107
zernike_coeffs[zernike_order] = 0.0001*wvln
global_zernike_aberrations.actuators = zernike_coeffs
zernike_aberrations = global_zernike_aberrations(wf_aper)
plt.figure(figsize=(10, 10))
hc.imshow_field(zernike_aberrations.phase)

In [ ]:
hc.make_gaussian_influence_functions?

In [ ]:
val

In [ ]:
mode = 0   # We start numbering at 0 here, 0-35 (Python nunmbering!)

# sm.flatten()
zernike_coeffs = np.zeros([mode_basis_global_zernike.num_modes])
for seg, val in enumerate(sorted_evecs[:, mode]):
    #print(seg)
    zernike_coeffs[seg*Nzernike] =  0.000000001*val/2
# Propagate WF and display SM phase
global_zernike_aberrations.actuators = zernike_coeffs
wf_sm = global_zernike_aberrations(wf_aper)

plt.figure(figsize=(10, 10))
hc.imshow_field(wf_sm.phase, cmap='RdBu')
plt.colorbar()

In [ ]:
# Lets compute all modes now for LUVOIR with an HCIPy SM
luvoir_modes = []

for mode in range(len(evals)):
    print('Working on mode {}/{}.'.format(mode+1, len(evals)))
    
    zernike_coeffs = np.zeros([mode_basis_global_zernike.num_modes])
    for seg, val in enumerate(sorted_evecs[:, mode]):
        #print(seg)
        zernike_coeffs[seg*Nzernike] =  0.000000001*val/2
# Propagate WF and display SM phase
    global_zernike_aberrations.actuators = zernike_coeffs
    wf_sm = global_zernike_aberrations(wf_aper)
    luvoir_modes.append(wf_sm.phase)

In [ ]:
# Lets compute all modes now for LUVOIR with an HCIPy SM
luvoir_modes = []

for mode in range(len(evals)):
    print('Working on mode {}/{}.'.format(mode+1, len(evals)))
    
    zernike_coeffs = np.zeros([mode_basis_global_zernike.num_modes])
    for seg, val in enumerate(sorted_evecs[:, mode]):
        #print(seg)
        zernike_coeffs[seg*Nzernike] =  0.000000001*val/2
# Propagate WF and display SM phase
    global_zernike_aberrations.actuators = zernike_coeffs
    wf_sm = global_zernike_aberrations(wf_aper)
    luvoir_modes.append(wf_sm.phase)

### Saving stuff

In [ ]:
# # Plot them all
# pmin = -5e-7
# pmax = 5e-7

# plt.figure(figsize=(24, 20))
# for mode in range(len(evals)):
    
#     plt.subplot(12, 10, mode+1)
#     hc.imshow_field(luvoir_modes[mode], cmap='RdBu')#, vmin=pmin, vmax=pmax)
#     #plt.colorbar()
#     plt.axis('off')
#     plt.title('Mode ' + str(mode+1))
    
# #plt.savefig(os.path.join(savedpath, 'results', 'LUVOIR_modes_piston.pdf'))

In [ ]:
# Plot them all
pmin = -5e-7
pmax = 5e-7

plt.figure(figsize=(24, 20))
for mode in range(len(evals)):
    
    plt.subplot(12, 10, mode+1)
    hc.imshow_field(luvoir_modes[mode], cmap='RdBu')#, vmin=pmin, vmax=pmax)
    #plt.colorbar()
    plt.axis('off')
    plt.title('Mode ' + str(mode+1))
    
#plt.savefig(os.path.join(savedpath, 'results', 'LUVOIR_modes_piston.pdf'))

In [ ]:
# Save them all to fits and PDF
all_modes = []   # to save as a cube

for mode in range(len(evals)):
    
    # fits
    #hc.write_fits(luvoir_modes[mode], os.path.join(savedpath, 'results', 'modes', 'fits', 'mode'+str(mode+1)+'.fits'))
    all_modes.append(luvoir_modes[mode].shaped)
    
    # pdf
    plt.clf()
    hc.imshow_field(luvoir_modes[mode], cmap='RdBu')
    plt.axis('off')
    plt.title('Mode ' + str(mode+1))
    #plt.savefig(os.path.join(savedpath, 'results', 'modes', 'pdf', 'mode'+str(mode+1)+'.pdf'))
    
# fits cube
all_modes = np.array(all_modes)
#hc.write_fits(all_modes, os.path.join(savedpath, 'results', 'modes', 'fits', 'cube_modes.fits'))
    
print('All done.')

### Do it with an SVD

`u` is holding all the modes in the form of `u[segment, mode]`. `s` holds all the singular values.

In [ ]:
u, s, vh = np.linalg.svd(matrix, full_matrices=True)

In [ ]:
print(s)

In [ ]:
#plt.plot(np.log10(s))
plt.plot(s)
plt.semilogy()
plt.ylabel('Log Eigenvalues')
plt.xlabel('Eigenmodes')

In [ ]:
print(s.shape)

As opposed to eigenvalues, the singular values are all positive, including the awkward global piston mode, and they're already sorted - **but from highest to lowest**! (opposite of above case)

In [ ]:
mode = -40   # We start numbering at 0 here, 0-35 (Python nunmbering!)

zernike_coeffs = np.zeros([mode_basis_global_zernike.num_modes])
for seg, val in enumerate(u[:, mode]):
    #print(val)
    zernike_coeffs[seg*Nzernike] =  0.000000001*val/2
# Propagate WF and display SM phase
global_zernike_aberrations.actuators = zernike_coeffs
wf_sm = global_zernike_aberrations(wf_aper)

hc.imshow_field(wf_sm.phase, cmap='RdBu')
plt.colorbar()

Plot them all because I don't know where the global piston is in the numbering from the SVD.

In [ ]:
plt.figure(figsize=(24, 20))
for mode in range(len(evals)):
    zernike_coeffs = np.zeros([mode_basis_global_zernike.num_modes])
    for seg, val in enumerate(u[:, mode]):
        zernike_coeffs[seg*Nzernike] =  0.000000001*val/2
        global_zernike_aberrations.actuators = zernike_coeffs
    wf_sm = global_zernike_aberrations(wf_aper)

    
    plt.subplot(12, 10, mode+1)
    hc.imshow_field(wf_sm.phase, cmap='RdBu')#, vmin=pmin, vmax=pmax)
    #plt.colorbar()
    plt.axis('off')
    plt.title('Mode ' + str(mode+1))

In [ ]:
plt.figure(figsize=(24, 20))
for mode in range(len(evals)):
    sm.flatten()
    for seg, val in enumerate(u[:, mode]):
        #print(val)
        sm.set_segment(seg+1, 0.000001*val/2, 0, 0)
        
    wf_sm = sm(wf_aper)
    
    plt.subplot(12, 10, mode+1)
    hc.imshow_field(wf_sm.phase, cmap='RdBu')#, vmin=pmin, vmax=pmax)
    #plt.colorbar()
    plt.axis('off')
    plt.title('Mode ' + str(mode+1))

Why on Earth is the global piston mode number 49 here?

In [ ]:
# Save them all to fits and PDF
all_modes = []   # to save as a cube

for mode in range(len(evals)):
    
    # fits
    #hc.write_fits(luvoir_modes[mode], os.path.join(savedpath, 'results', 'modes', 'fits', 'mode'+str(mode+1)+'.fits'))
    all_modes.append(luvoir_modes[mode].shaped)
    
    # pdf
    plt.clf()
    hc.imshow_field(luvoir_modes[mode], cmap='RdBu')
    plt.axis('off')
    plt.title('Mode ' + str(mode+1))
    #plt.savefig(os.path.join(savedpath, 'results', 'modes', 'pdf', 'mode'+str(mode+1)+'.pdf'))
    
# fits cube
all_modes = np.array(all_modes)
#hc.write_fits(all_modes, os.path.join(savedpath, 'results', 'modes', 'fits', 'cube_modes.fits'))
    
print('All done.')